In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
with open('train.csv', 'w') as file3:
    with open('y_train.txt', 'r') as file1:
        with open('X_train.txt', 'r') as file2:
            for line1, line2 in zip(file1, file2):
                print(line1.strip(), line2.strip(), file=file3)

In [ ]:
with open('devnb.csv', 'w') as file3:
    with open('y_dev.txt', 'r') as file1:
        with open('X_dev.txt', 'r') as file2:
            for line1, line2 in zip(file1, file2):
                print(line1.strip(), line2.strip(), file=file3)

In [79]:
import re
def tokenize(text):
    temp = text.split(" ")
    return temp
def better_tokenize(text): 
    temp = text.split(" ")
    temp = [(word.lower()) for word in temp]
    temp = [(re.sub('[^A-Za-z]+', '', word)) for word in temp]
    
    words_to_be_removed=["having",'didnt','doesnt','n','u','ur','i','I',"youre","im","a","an", "about", "above", "across", "after", "afterwards", "again", "against", "all", "almost", "alone", "along", "already", "also","although","always","am","among", "amongst", "amoungst", "amount", "an", "and", "another", "any","anyhow","anyone","anything","anyway", "anywhere", "are", "around", "as", "at", "back","be","became", "because","become","becomes", "becoming", "been", "before", "beforehand", "behind", "being", "below", "beside", "besides", "between", "beyond", "bill", "both", "bottom","but", "by", "call", "can", "cannot", "cant", "co", "con", "could", "couldnt", "cry", "de", "describe", "detail", "do", "done", "down", "due", "during", "each", "eg", "eight", "either", "eleven","else", "elsewhere", "empty", "enough", "etc", "even", "ever", "every", "everyone", "everything", "everywhere", "except", "few", "fifteen", "fify", "fill", "find", "fire", "first", "five", "for", "former", "formerly", "forty", "found", "four", "from", "front", "full", "further", "get", "give", "go", "had", "has", "hasnt", "have", "he", "hence", "her", "here", "hereafter", "hereby", "herein", "hereupon", "hers", "herself", "him", "himself", "his", "how", "however", "hundred","i" "ie", "if", "in", "inc", "indeed", "interest", "into", "is", "it", "its", "itself", "keep", "last", "latter", "latterly", "least", "less", "ltd", "made", "many", "may", "me", "meanwhile", "might", "mill", "mine", "more", "moreover", "most", "mostly", "move", "much", "must", "my", "myself", "name", "namely", "neither", "never", "nevertheless", "next", "nine", "no", "nobody", "none", "noone", "nor", "not", "nothing", "now", "nowhere", "of", "off", "often", "on", "once", "one", "only", "onto", "or", "other", "others", "otherwise", "our", "ours", "ourselves", "out", "over", "own","part", "per", "perhaps", "please", "put", "rather", "re", "same", "see", "seem", "seemed", "seeming", "seems", "several", "she", "should", "show", "since", "six", "sixty", "so", "some", "somehow", "someone", "something", "sometime", "sometimes", "somewhere", "still", "such", "system", "take", "ten", "than", "that","thats", "the", "their", "them", "themselves", "then", "thence", "there", "thereafter", "thereby", "therefore", "therein", "thereupon", "these", "they", "thickv", "thin", "third", "this", "those", "though", "three", "through", "throughout", "thru", "thus", "to", "too", "top", "toward", "towards", "twelve", "twenty", "two", "un", "under", "until", "up", "upon", "us", "very", "via", "was", "we", "well", "were", "what", "whatever", "when", "whence", "whenever", "where", "whereafter", "whereas", "whereby", "wherein", "whereupon", "wherever", "whether", "which", "while", "whither", "who", "whoever", "whole", "whom", "whose", "why", "will", "with", "within","without", "would", "yet", "you", "your", "yours", "yourself", "yourselves", "the"]
    
    for j in range(0,len(temp)):
        for i in range(0,len(words_to_be_removed)):
            if(temp[j]==words_to_be_removed[i]):
                temp[j]=''
    

    temp=[x for x in temp if x]
    return temp

In [80]:
better_tokenize("Awwww....DUDE. That sucks!! Hope you find a paperclip and a rubberband!!")

['awwwwdude', 'sucks', 'hope', 'paperclip', 'rubberband']

In [81]:
import csv
from collections import defaultdict
from collections import Counter

troll_dictionary={}
normal_dictionary={}
troll_word_to_prob={}
normal_word_to_prob={}
troll_prob=0
normal_prob=0
accuracy=0

In [82]:
def laplace_smoothing(counter_name, alpha=0):
    word_to_count = counter_name.copy() ## Make sure you use a "copy()" not to affect your original dictionary
    V = len(word_to_count) ## V means the number of unique words (== a size of vocab)
    #print(V)
    for w, c in word_to_count.items():
        word_to_count[w] += alpha  ## == (n_ij + alpha) == numerator
    word_to_prob = {}  ## build a new dictionary for new values after smoothing
    total_counts = float(sum(word_to_count.values()))     
    ##Since we've added alpha to all of the frequencies, total_counts implicitly is sum_freq + alpha*V
    for w, c in word_to_count.items():
        word_to_prob[w] = c / total_counts
    return word_to_prob

In [83]:
def train_classify(smoothing_alpha=0):
    with open('train.csv', mode='r') as infile:
        reader = list(csv.reader(infile))
        header = reader[0]
        troll=0
        normal=0
        total_count=0
        troll_list=[]
        normal_list=[]
        for row in reader[1:]:
            if(row[0]=='1'):
                troll = troll +1
                total_count=total_count+1
                tweet= row[1]
                troll_list=troll_list+better_tokenize(tweet)
                tweet
        
                
            else:
                normal =normal +1
                total_count= total_count+1
                tweet=row[1]
                normal_list=normal_list+better_tokenize(tweet)
        troll_dictionary= Counter(troll_list)
        normal_dictionary=Counter(normal_list)
        #print(normal_dictionary)
        
        troll_word_to_prob = laplace_smoothing(troll_dictionary,smoothing_alpha)
        #print(troll_word_to_prob)
        normal_word_to_prob = laplace_smoothing(normal_dictionary,smoothing_alpha)
        troll_prob= troll/total_count
        normal_prob= normal/total_count
        print(len(troll_dictionary))
        print("----------------")
        print(len(normal_dictionary))
        print("----------------")
        print(troll_prob)
        print("----------------")
        print(normal_prob)
        print("----------------")
        print(len(troll_word_to_prob))
        print("----------------")
        print(len(normal_word_to_prob))
        print("----------------")
    infile.close()
    with open('classify-output1.csv', 'w', newline='') as file2:
        fieldnames=['Troll','Tweet']
        writer=csv.DictWriter(file2, fieldnames=fieldnames)
        writer.writeheader()
        with open('testnb.csv') as infile:
            reader = list(csv.reader(infile))
            header = reader[0]
            accuracy=0
            for row in reader[1:]:
                prob1=1
                prob2=1
                tokens=better_tokenize(row[1])
                #print(tokens)
                for tok in troll_dictionary:
                    if tok in tokens:
                        prob1=prob1*troll_word_to_prob[tok]*tokens.count(tok)
                    else:
                        prob1=prob1*(1-troll_word_to_prob[tok])
                for tok in normal_dictionary:
                    if tok in tokens:
                        prob2=prob2*normal_word_to_prob[tok]*tokens.count(tok)
                    else:
                        prob2=prob2*(1-normal_word_to_prob[tok])
            
                troll_out= prob1*troll_prob/(prob1*troll_prob+prob2*normal_prob+0.0000000000001)
                normal_out= prob2*normal_prob/(prob1*troll_prob+prob2*normal_prob+0.0000000000001)
                if(troll_out>0.5):
                    writer.writerow({'Troll':'1','Tweet':row[1]})
                else:
                    writer.writerow({'Troll':'0','Tweet':row[1]})

In [84]:
train_classify(0)

7085
----------------
9565
----------------
0.3887
----------------
0.6113
----------------
7085
----------------
9565
----------------


In [85]:
def train_classify_dev(smoothing_alpha=0):
    with open('train.csv', mode='r') as infile:
        reader = list(csv.reader(infile))
        header = reader[0]
        troll=0
        normal=0
        total_count=0
        troll_list=[]
        normal_list=[]
        for row in reader[1:]:
            if(row[0]=='1'):
                troll = troll +1
                total_count=total_count+1
                tweet= row[1]
                troll_list=troll_list+better_tokenize(tweet)
                #print(type(troll_list))
        
                
            else:
                normal =normal +1
                total_count= total_count+1
                tweet=row[1]
                normal_list=normal_list+better_tokenize(tweet)
        troll_dictionary= Counter(troll_list)
        normal_dictionary=Counter(normal_list)
        print(normal_dictionary)
        
        troll_word_to_prob = laplace_smoothing(troll_dictionary,smoothing_alpha)
        #print(troll_word_to_prob)
        normal_word_to_prob = laplace_smoothing(normal_dictionary,smoothing_alpha)
        troll_prob= troll/total_count
        normal_prob= normal/total_count
        print(len(troll_dictionary))
        print("----------------")
        print(len(normal_dictionary))
        print("----------------")
        print(troll_prob)
        print("----------------")
        print(normal_prob)
        print("----------------")
        print(len(troll_word_to_prob))
        print("----------------")
        print(len(normal_word_to_prob))
        print("----------------")
    infile.close()
    with open('classify-output2.csv', 'w', newline='') as file2:
        fieldnames=['troll','Tweet']
        writer=csv.DictWriter(file2, fieldnames=fieldnames)
        writer.writeheader()
        with open('devnb.csv') as infile:
            reader = list(csv.reader(infile))
            header = reader[0]
            accuracy=0
            for row in reader[1:]:
                prob1=1
                prob2=1
                tokens=better_tokenize(row[1])
                #print(tokens)
                for tok in troll_dictionary:
                    if tok in tokens:
                        prob1=prob1*troll_word_to_prob[tok]*tokens.count(tok)
                    else:
                        prob1=prob1*(1-troll_word_to_prob[tok])
                for tok in normal_dictionary:
                    if tok in tokens:
                        prob2=prob2*normal_word_to_prob[tok]*tokens.count(tok)
                    else:
                        prob2=prob2*(1-normal_word_to_prob[tok])
            
                troll_out= prob1*troll_prob/(prob1*troll_prob+prob2*normal_prob+0.0000000000001)
                normal_out= prob2*normal_prob/(prob1*troll_prob+prob2*normal_prob+0.0000000000001)
                #print(troll_out,normal_out)
                if(normal_out<troll_out):
                    if(row[0]=='1'):
                        accuracy+=1
                else:
                    if(row[0]=='0'):
                        accuracy+=1
            
        print(accuracy)

In [86]:
train_classify_dev(1)

Counter({'hate': 846, 'damn': 722, 'ass': 537, 'sucks': 502, 'fuck': 487, 'like': 449, 'lol': 428, 'just': 398, 'dont': 350, 'fucking': 285, 'know': 273, 'love': 234, 'think': 224, 'bitch': 196, 'people': 187, 'fat': 186, 'say': 181, 'haha': 176, 'good': 171, 'want': 158, 'really': 155, 'yeah': 154, 'gay': 153, 'yes': 147, 'got': 147, 'oh': 143, 'time': 136, 'man': 122, 'make': 113, 'shit': 113, 'did': 110, 'thing': 109, 'right': 108, 'r': 106, 'going': 105, 'need': 105, 'way': 100, 'work': 99, 'ill': 98, 'id': 95, 'day': 91, 'ive': 91, 'life': 90, 'd': 90, 'bad': 86, 'hes': 85, 'whats': 83, 'feel': 81, 'dude': 78, 'pretty': 75, 'big': 74, 'tell': 74, 'thanks': 73, 'better': 72, 'sorry': 71, 'sure': 69, 'new': 69, 'p': 69, 'hope': 68, 'dick': 66, 'does': 63, 'nerd': 63, 'best': 63, 'kick': 62, 'look': 60, 'twitter': 60, 'freak': 60, 'scrabble': 60, 'hard': 59, 'maybe': 59, 'night': 58, 'lmao': 57, 'mean': 57, 'ya': 56, 'guess': 55, 'cause': 54, 'stop': 54, 'gonna': 54, 'guy': 53, 'haha

1061
